In [ ]:
import os

import numpy as np
import pandas as pd

from scipy.stats import fisher_exact
from statsmodels.sandbox.stats.multicomp import multipletests

In [ ]:
project_directory = '/Users/tstoeger/Dropbox/biomedical_data_science_day/'  # <-- modify

In [ ]:
def complete_path(p):
    """
    Completes the path toward a file.
    
    Input:
        p     str, path relative to project path
    
    Output:
        p     str, complete path
    """
    p = os.path.join(project_directory, p)
    return p

In [ ]:
# def read_go_and_turn_to_annotation(taxon_id, subset):
# """
# Reads Gene Ontology (GO) annotation and formats them
# toward format anticipated by our enrichment tool.

# Input:
#     taxon_id    int, NCBI taxonomy identifier 
#                      (e.g.: 9606 for human)
#     subset      int, 'Component', 'Process', 
#                     or 'Function', or 'all'

# Output:
#     gene_annotation  df, with columns
#                      'gene_entrez' and 'annotation' where
#                      'gene_entrez': int with Entrez Gene Identifier
#                      'annotation': str with name of annotation
# """

# load tab-delimited file
p = complete_path('ncbi_gene/gene2go.gz')   

df = #...... inset

# filter according to taxon
df = df.loc[df.loc[:, '#tax_id']==taxon_id, :]

# filter according to sub-category of GO

#...... inset

# Exclude annotations for which there is some 
# negative evidence (e.g.: annotators found that
# a plausible annotation actually does not occur)
# Note that for a given pair of gene and annotation
# there may be positive and negative evidence, but
# all pairs should be removed if there is any negative
# http://geneontology.org/docs/guide-go-evidence-codes/

#...... inset

# Get future pairs of  'gene_entrez' and 'annotation',
# while removing duplicates; and format them according
# to the docstring (help/description ) of this function
# 'GeneID' will become 'gene_entrez', and 'GO_term'
# will become 'annotation'

df = df.loc[:, ['GeneID', 'GO_term']].drop_duplicates()

df = df.rename(columns={
    'GeneID': 'gene_entrez', 
    'GO_term': 'annotation'}).reset_index(drop=True)
df['gene_entrez'] = df['gene_entrez'].astype(int)

#     return df

In [ ]:
taxon_id = 9606   # the NCBI taxonomy identifier for Homo sapiens
gene_annotation = read_go_and_turn_to_annotation(taxon_id, 'all')

In [ ]:
gene_annotation.head()

In [ ]:
# def functional_enrichment(
#     hits, 
#     background, 
#     gene_annotation,
#     only_genes_with_annotation=True
# ):
# """
# Tool to compute functional enrichments using Fisher's
# Exact test.

# Input:
#     hits         list-like, gene_entrez of hits
#     background   list-like, gene_entrez of background (all genes in experiment)
#     gene_annotation   df, with columns
#                      'gene_entrez' and 'annotation' where
#                      'gene_entrez': int with Entre Gene Identifier
#                      'annotation': str with name of annotation 
#     only_genes_with_annotation  optional, default: True
#                                 if True, hit and backround
#                                 will be filtered for genes
#                                 in gene_annotation

# Output:
#     results  df, with index 'annotation' and columns
#                 'fold_enrichment' log2 ratio of enrichment of annotation over backround
#                 'pvalue' Fisher's exact test on enrichment of annotation
#                 'bonferroni' Bonferroni corrected pvalue (multiple testing)
#                 'benjamini_hochberg' Benjamini Hochberg corrected pvalue (multiple testing)
# """


# Create three sets:
# - hits: the hits from the study
# - background: the user supplied list of considered genes
# - not hits: genes in background but not hits

#...... inset

# Optionally, use filter to only consider genes
# that have at least one annotation, e.g.: see
# https://www.nature.com/articles/s41598-018-19333-x
# on gene annotation bias
if only_genes_with_annotation:
    unique_annotations = set(gene_annotation['gene_entrez'])

    hits = hits.intersection(unique_annotations)
    background = background.intersection(unique_annotations)
    not_hits = not_hits.intersection(unique_annotations)

# Force an error if not all hits are in the background
if any([hit not in background for hit in hits]):
    raise AssertionError(
        'At least one hit is not in the supplied background.'
    )

# Restrict annotations to those that occur in the background
gene_annotation = #...... inset

# Restrict annotations to those occuring a certain number
# Rationale is to get rid of the bulk of annotations that only affect
# a very small number of gens <-- note this is somewhat arbritrary
# and subjective cutoff
minimally_required_candidates_in_annotation = 2
c = gene_annotation['annotation'].value_counts()
gene_annotation = gene_annotation[gene_annotation['annotation'].isin(
    c[c >= minimally_required_candidates_in_annotation].index)]

# Prepare to loop through annotations
# First, set 'annotation' to become the index of gene_annotation
gene_annotation = #...... inset
# Second, get all unique 'annotation'
annotations = #...... inset
# Third, initialize a dataframe with all unique annotations in
# rows, and 'fold_enrichment', 'pvalue', 'bonferroni',
# 'benjamini_hochberg' as columns
results = #...... inset

# Loop through annotations
for annotation in annotations:

    # Set up a contingency table containing 
    #   [hit_genes_in_annotation, not_hit_genes_in_annotation], 
    #   [hit_genes_not_in_annotation, not_hit_genes_not_in_annotation]
    # where each of above is total number of such genes
    # https://www.pathwaycommons.org/guide/primers/statistics/fishers_exact_test/

    genes_with_annotation = #...... inset
    genes_without_annotation = #...... inset

    hit_genes_in_annotation = #...... inset
    not_hit_genes_in_annotation = #...... inset

    hit_genes_not_in_annotation = #...... inset
    not_hit_genes_not_in_annotation = #...... inset

    # Perform Fisher's exact test
    _, pvalue = fisher_exact(
        [
             [hit_genes_in_annotation, not_hit_genes_in_annotation], 
             [hit_genes_not_in_annotation, not_hit_genes_not_in_annotation]],
        alternative='greater'
    )

    # Count number of annotated genes in hits and backround and compute
    # the enrichment among hits
    in_hits = #...... inset
    in_background = #...... inset

    fract_of_hits_w_annot = #...... inset
    fract_of_background_w_annot = #...... inset

    if fract_of_hits_w_annot>0:
        enrichment = np.log2(
            fract_of_hits_w_annot / fract_of_background_w_annot)
    else:
        enrichment = -np.inf

    results.loc[annotation, 'fold_enrichment'] = enrichment
    results.loc[annotation, 'pvalue'] = pvalue

# Add corrections for multiple testing
results.loc[:, 'bonferroni'] = #...... inset
results.loc[:, 'benjamini_hochberg'] = #...... inset

# Sort results by pvalue
results = results.sort_values('pvalue')

#     return results

In [ ]:
# Get list of hits and background genes to be considered (all in experiment)
# in this case we want to know, which annotations are carried by the 
# genes that most commonly appear in differential gene expression analysis
# Data is from Crow et al. 2019 "Predictablity of human gene expression",
# PNAS, https://www.pnas.org/content/116/13/6491


p = complete_path('candidate_genes/crow_et_al_2019/DE_Prior.txt')
study_with_genes = pd.read_csv(p, sep='\t')  # a table that sorts genes according to occurence
background_from_study = study_with_genes['Gene_EntrezID']
hits_from_study = study_with_genes.loc[:300, 'Gene_EntrezID']  # lets' treat the 300 most frequent genes as hits

In [ ]:
go_enrichments = functional_enrichment(
    hits=hits_from_study, 
    background=background_from_study, 
    gene_annotation=gene_annotation,
    only_genes_with_annotation=False
)

In [ ]:
go_enrichments.head()

In [ ]:
# def read_gwas_and_turn_to_annotation():
#     """
#     Reads EBI GWAS annotation and formats them
#     toward format anticipated by our enrichment tool.
    
#     Output:
#         gene_annotation  df, with columns
#                          'gene_entrez' and 'annotation' where
#                          'gene_entrez': int with Entrez Gene Identifier
#                          'annotation': str with name of trait
#     """
    
#     p = complete_path('ebi_gwas/full.tsv')
#     df_gwas = pd.read_csv(
#         p, 
#         sep='\t', 
#         usecols=['DISEASE/TRAIT', 'MAPPED_GENE']
#     ).drop_duplicates()

#     p = complete_path('ncbi_gene/Homo_sapiens.gene_info.gz')
#     df_gene_info = pd.read_csv(p, sep='\t', usecols=['GeneID', 'Symbol'])

#     # combine overly simplistically through merge: unambiguous single SNPs 
#     # -> only for illustrative purposes of tutorial
#     df_annotation = pd.merge(
#         df_gene_info.rename(columns={'Symbol': 'MAPPED_GENE'}),
#         df_gwas,
#         how='inner'   
#     ).rename(columns={
#         'GeneID': 'gene_entrez',
#         'DISEASE/TRAIT': 'annotation'})

#     df_annotation = df_annotation[['gene_entrez', 'annotation']].drop_duplicates()

#     return df_annotation

In [ ]:
# gene_annotation = read_gwas_and_turn_to_annotation()

In [ ]:
# gwas_enrichments = functional_enrichment(
#     hits=hits_from_study, 
#     background=background_from_study, 
#     gene_annotation=gene_annotation,
#     only_genes_with_annotation=False
# )

In [ ]:

# gwas_enrichments.head()